<a href="https://colab.research.google.com/github/Mikhail-068/NetOptic/blob/master/Tretyakov_Aleksandr/3_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1. На основе имеющегося датасета с изображениями очков из различных материалов загрузим выборки с использованием генератора.
2. Проведём аугментацию про помощи Imagedatagenerator.
3. Найдём наилучшую модель при помощи библиотеки AutoKeras и KerasTuner.

In [1]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.4 MB/s eta 0:00:00


In [2]:
# загрузка библиотек
import os
import gdown
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

In [3]:
# загрузка датасета, укажем путь к базе в Google Drive, база в виде .zip-архива
URL = 'https://drive.google.com/uc?id=17g0sp2Xt27ONyzJ9667HroY6010_Tli_'
gdown.download(URL)

Downloading...
From: https://drive.google.com/uc?id=17g0sp2Xt27ONyzJ9667HroY6010_Tli_
To: /content/materials_pictures_archive.zip
100%|██████████| 116M/116M [00:02<00:00, 51.7MB/s]


'materials_pictures_archive.zip'

In [4]:
!unzip -qo 'materials_pictures_archive.zip' -d /content/materials_pictures/

In [5]:
IMAGE_PATH = '/content/materials_pictures/'
print(os.listdir(IMAGE_PATH))

['металл', 'комбинированный', 'пластик']


In [6]:
# IMG_HEIGHT, IMG_WIDTH = 600, 600
IMG_HEIGHT, IMG_WIDTH = 60, 60

In [7]:
# Создаем генератор для аугментации
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # диапазон углов для вращения изображений
    width_shift_range=0.2,  # диапазон, который используется для горизонтального смещения изображения
    height_shift_range=0.2, # диапазон, который используется для вертикального смещения изображения
    zoom_range=0.2,         # диапазон для случайного масштабирования изображения
    fill_mode='nearest',    # стратегия заполнения появившихся пустых мест при преобразованиях
    validation_split=0.2)

In [8]:
# валидационную выборку только нормируем, к ней не будет применена аугментация
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [9]:
# Создаём генераторы, загружающие картинки с диска

train_data = train_datagen.flow_from_directory(
    IMAGE_PATH, # путь до вашего датасета
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_data = validation_datagen.flow_from_directory(
    IMAGE_PATH, # путь до вашего датасета
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 4151 images belonging to 3 classes.
Found 1036 images belonging to 3 classes.


In [10]:
NUM_CLASSES = 3

In [11]:
# определяем архитектуру модели, которую будем подавать в KerasTuner
def model_builder(hp):
    model = Sequential()

    # Сверточный слой
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

    # Добавляем больше сверточных слоев
    for i in range(hp.Int('conv_layers', 1, 3)):
        model.add(layers.Conv2D(hp.Choice(f'conv_units_{i}', [32, 64, 128]), (3, 3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    # Добавляем полносвязные слои
    for i in range(hp.Int('dense_layers', 1, 2)):
        model.add(layers.Dense(hp.Choice(f'dense_units_{i}', [64, 128, 256]), activation='relu'))
        model.add(layers.Dropout(hp.Choice(f'dropout_{i}', [0.1, 0.2, 0.3, 0.4, 0.5])))

    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ['accuracy'])

    return model

In [12]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 4,
                     directory = 'tuner_result',
                     project_name = 'net_optics')

In [13]:
tuner.search(train_data,
             epochs = 10,
             validation_data = val_data)

Trial 10 Complete [00h 02m 03s]
val_accuracy: 0.4536679685115814

Best val_accuracy So Far: 0.5231660008430481
Total elapsed time: 00h 13m 27s


In [14]:
best_model = tuner.get_best_models(num_models=1)[0]

In [15]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 128)       36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 32)        36896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 7, 7, 32)         0

In [16]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
pprint(best_hyperparameters.values)

{'conv_layers': 2,
 'conv_units_0': 128,
 'conv_units_1': 32,
 'conv_units_2': 128,
 'dense_layers': 2,
 'dense_units_0': 64,
 'dense_units_1': 64,
 'dropout_0': 0.3,
 'dropout_1': 0.1,
 'learning_rate': 0.001,
 'tuner/bracket': 1,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0000'}
